## Librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch
import pickle as pkl

import warnings
warnings.filterwarnings('ignore')

## Extraccion de datos

In [2]:
df_events = pd.read_csv('D:\Futbol\Analisis presion alata en PL 23-24\data\pl_23-24_events.csv')

equipos = \
{
13: 'Arsenal',
15: 'Chelsea',
23: 'Newcastle Utd',
24: 'Aston Villa',
26: 'Liverpool',
29: 'West Ham',
30: 'Tottenham',
31: 'Everton',
32: 'Manchester Utd',
95: 'Luton Town',
161: 'Wolves',
162: 'Crystal Palace',
163: 'Sheffield Utd',
167: 'Manchester City',
170: 'Fulham',
174: "Nott'ham Forest",
183: 'Bournemouth',
184: 'Burnley',
189: 'Brentford',
211: 'Brighton'
}

df_equipos = pd.DataFrame(list(equipos.items()), columns=['teamId', 'Equipo'])

In [12]:
df_events.columns.to_list()

['Unnamed: 0',
 'id',
 'eventId',
 'minute',
 'second',
 'teamId',
 'h_a',
 'x',
 'y',
 'expandedMinute',
 'period',
 'type',
 'outcomeType',
 'qualifiers',
 'satisfiedEventsTypes',
 'isTouch',
 'matchId',
 'startDate',
 'startTime',
 'score',
 'ftScore',
 'htScore',
 'etScore',
 'venueName',
 'maxMinute',
 'playerId',
 'playerName',
 'endX',
 'endY',
 'relatedEventId',
 'relatedPlayerId',
 'goalMouthZ',
 'goalMouthY',
 'isGoal',
 'isShot',
 'blockedX',
 'blockedY',
 'cardType',
 'shotBodyType',
 'situation',
 'shotSixYardBox',
 'shotPenaltyArea',
 'shotOboxTotal',
 'shotOpenPlay',
 'shotCounter',
 'shotSetPiece',
 'shotDirectCorner',
 'shotOffTarget',
 'shotOnPost',
 'shotOnTarget',
 'shotsTotal',
 'shotBlocked',
 'shotRightFoot',
 'shotLeftFoot',
 'shotHead',
 'shotObp',
 'goalSixYardBox',
 'goalPenaltyArea',
 'goalObox',
 'goalOpenPlay',
 'goalCounter',
 'goalSetPiece',
 'penaltyScored',
 'goalOwn',
 'goalNormal',
 'goalRightFoot',
 'goalLeftFoot',
 'goalHead',
 'goalObp',
 'shortPa

In [9]:
df_events['type'].unique()

array(['Start', 'Pass', 'Aerial', 'BallRecovery', 'Clearance',
       'BlockedPass', 'Dispossessed', 'Tackle', 'Foul', 'TakeOn',
       'BallTouch', 'ShieldBallOpp', 'CornerAwarded', 'Goal',
       'KeeperSweeper', 'MissedShots', 'KeeperPickup', 'Interception',
       'OffsideGiven', 'OffsidePass', 'OffsideProvoked', 'SavedShot',
       'Save', 'SubstitutionOff', 'SubstitutionOn', 'FormationChange',
       'Challenge', 'Error', 'Smother', 'End', 'Claim', 'Card',
       'FormationSet', 'Punch', 'ShotOnPost', 'PenaltyFaced', 'GoodSkill',
       'ChanceMissed', 'CrossNotClaimed'], dtype=object)

## Tackles

In [3]:
df_tackle = df_events[(df_events['type'] == 'Tackle')][['eventId', 'matchId', 'teamId', 'minute', 'second', 'h_a', 'x', 'y', 'tackleLastMan', 'tackleLost', 'tackleWon']]

In [4]:
tackle_def3rd = 0 + (100 - 0) / 3
tackle_mid3rd = tackle_def3rd + (100 - 0) / 3

df_tackle['tackle_def3rd'] = df_tackle['x'] <= tackle_def3rd
df_tackle['tackle_mid3rd'] = (df_tackle['x'] > tackle_def3rd) & (df_tackle['x'] < tackle_mid3rd)
df_tackle['tackle_att3rd'] = df_tackle['x'] >= tackle_mid3rd
df_tackle['tackle_won_def3rd'] = df_tackle.apply(lambda row: row['tackleWon'] and row['tackle_def3rd'], axis=1)
df_tackle['tackle_won_mid3rd'] = df_tackle.apply(lambda row: row['tackleWon'] and row['tackle_mid3rd'], axis=1)
df_tackle['tackle_won_att3rd'] = df_tackle.apply(lambda row: row['tackleWon'] and row['tackle_att3rd'], axis=1)

In [8]:
df_tackle_g = df_tackle.groupby('teamId').agg(tackle=('eventId', 'count'),
                                              tackle_won=('tackleWon', lambda x: x.sum()),
                                              tackle_lost=('tackleLost', lambda x: x.sum()),
                                              tackle_lastman=('tackleLastMan', lambda x: x.sum()),
                                              tackle_def3rd=('tackle_def3rd', lambda x: x.sum()),
                                              tackle_mid3rd=('tackle_mid3rd', lambda x: x.sum()),
                                              tackle_att3rd=('tackle_att3rd', lambda x: x.sum()),
                                              tackle_won_def3rd=('tackle_won_def3rd', lambda x: x.sum()),
                                              tackle_won_mid3rd=('tackle_won_mid3rd', lambda x: x.sum()),
                                              tackle_won_att3rd=('tackle_won_att3rd', lambda x: x.sum())
                                             ).reset_index()

In [7]:
df_tackle_stats = pd.merge(df_tackle_g, df_equipos, on='teamId')
df_tackle_stats[['teamId', 'Equipo', 'tackle', 'tackle_won', 'tackle_lost', 'tackle_lastman',
                 'tackle_def3rd', 'tackle_mid3rd', 'tackle_att3rd', 'tackle_won_def3rd',
                 'tackle_won_mid3rd', 'tackle_won_att3rd']]

,teamId,Equipo,tackle,tackle_won,tackle_lost,tackle_lastman,tackle_def3rd,tackle_mid3rd,tackle_att3rd,tackle_won_def3rd,tackle_won_mid3rd,tackle_won_att3rd
0,13,Arsenal,610,348,262,6,257,246,107,147,145,56
1,15,Chelsea,663,376,287,3,325,235,103,192,133,51
2,23,Newcastle Utd,661,391,270,6,316,254,91,185,153,53
3,24,Aston Villa,547,311,236,5,269,210,68,150,118,43
4,26,Liverpool,677,418,259,9,274,290,113,174,176,68
5,29,West Ham,684,426,258,4,350,254,80,203,170,53
6,30,Tottenham,724,451,273,10,335,264,125,210,167,74
7,31,Everton,743,442,301,10,371,261,111,229,149,64
8,32,Manchester Utd,676,409,267,5,334,240,102,200,148,61
9,95,Luton Town,654,368,286,5,330,236,88,191,132,45


In [13]:
with open('D:\Futbol\Analisis presion alata en PL 23-24\pkl\df_tackle_stats.pkl', 'wb') as archivo:
    pkl.dump(df_tackle_stats, archivo, protocol=pkl.HIGHEST_PROTOCOL)

In [14]:
with open('D:\Futbol\Analisis presion alata en PL 23-24\pkl\df_tackle_stats.pkl', 'rb') as archivo:
    df_tackle_stats = pkl.load(archivo)

## Intercepciones

In [7]:
df_int = df_events[(df_events['type'] == 'Interception')][['eventId', 'matchId', 'teamId', 'minute', 'second', 'h_a', 'x', 'y', 'interceptionAll', 'interceptionIntheBox', 'interceptionWon']]

In [9]:
int_def3rd = 0 + (100 - 0) / 3
int_mid3rd = int_def3rd + (100 - 0) / 3

df_int['int_def3rd'] = df_int['x'] <= int_def3rd
df_int['int_mid3rd'] = (df_int['x'] > int_def3rd) & (df_int['x'] < int_mid3rd)
df_int['int_att3rd'] = df_int['x'] >= int_mid3rd
df_int['int_won_def3rd'] = df_int.apply(lambda row: row['interceptionWon'] and row['int_def3rd'], axis=1)
df_int['int_won_mid3rd'] = df_int.apply(lambda row: row['interceptionWon'] and row['int_mid3rd'], axis=1)
df_int['int_won_att3rd'] = df_int.apply(lambda row: row['interceptionWon'] and row['int_att3rd'], axis=1)

In [14]:
df_int_g = df_int.groupby('teamId').agg(int=('eventId', 'count'),
                                              int_won=('interceptionWon', lambda x: x.sum()),
                                              int_def3rd=('int_def3rd', lambda x: x.sum()),
                                              int_mid3rd=('int_mid3rd', lambda x: x.sum()),
                                              int_att3rd=('int_att3rd', lambda x: x.sum()),
                                              int_won_def3rd=('int_won_def3rd', lambda x: x.sum()),
                                              int_won_mid3rd=('int_won_mid3rd', lambda x: x.sum()),
                                              int_won_att3rd=('int_won_att3rd', lambda x: x.sum())
                                             ).reset_index()

In [15]:
df_int_g

,teamId,int,int_won,int_def3rd,int_mid3rd,int_att3rd,int_won_def3rd,int_won_mid3rd,int_won_att3rd
0,13,292,292,103,139,50,103,139,50
1,15,298,298,132,128,38,132,128,38
2,23,285,285,132,126,27,132,126,27
3,24,236,236,115,97,24,115,97,24
4,26,292,292,101,156,35,101,156,35
5,29,361,361,195,134,32,195,134,32
6,30,355,355,135,162,58,135,162,58
7,31,358,358,167,146,45,167,146,45
8,32,313,313,171,111,31,171,111,31
9,95,325,325,163,146,16,163,146,16


In [17]:
df_int_stats = pd.merge(df_int_g, df_equipos, on='teamId')
df_int_stats[['teamId', 'Equipo', 'int', 'int_won', 'int_def3rd',
              'int_mid3rd', 'int_att3rd', 'int_won_def3rd',
              'int_won_mid3rd', 'int_won_att3rd']]

,teamId,Equipo,int,int_won,int_def3rd,int_mid3rd,int_att3rd,int_won_def3rd,int_won_mid3rd,int_won_att3rd
0,13,Arsenal,292,292,103,139,50,103,139,50
1,15,Chelsea,298,298,132,128,38,132,128,38
2,23,Newcastle Utd,285,285,132,126,27,132,126,27
3,24,Aston Villa,236,236,115,97,24,115,97,24
4,26,Liverpool,292,292,101,156,35,101,156,35
5,29,West Ham,361,361,195,134,32,195,134,32
6,30,Tottenham,355,355,135,162,58,135,162,58
7,31,Everton,358,358,167,146,45,167,146,45
8,32,Manchester Utd,313,313,171,111,31,171,111,31
9,95,Luton Town,325,325,163,146,16,163,146,16


In [19]:
with open('D:\Futbol\Analisis presion alata en PL 23-24\pkl\df_int_stats.pkl', 'wb') as archivo:
    pkl.dump(df_int_stats, archivo, protocol=pkl.HIGHEST_PROTOCOL)